In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import logging
import pandas as pd
import re
import time
import json
import xlsxwriter

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [70]:
options = webdriver.ChromeOptions()
options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

In [121]:
driver = webdriver.Chrome(service=Service(executable_path="/opt/homebrew/bin/chromedriver"), options=options)

driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="6b82a0979fac8c9dcc3350794067bb51")>

In [122]:
table_rows = driver.find_elements(By.TAG_NAME, 'tr')
type(table_rows)
len(table_rows)

# table_rows


# table_rows[0].text
# table_rows[0]
# header = '<selenium.webdriver.remote.webelement.WebElement (session="9ce3ac1101a0c1a1e89cdad93bec4408", element="04DBA8B80C068AF17A38D8C62CF98ED1_element_93")>'
# 
# 
# table_rows_minus_header = table_rows[1:]
# 
# if header in table_rows_minus_header:
#     print('true')
# else:
#     print('false')


13

In [123]:
def get_all_table_rows_except_header_row():
    try:
        table_rows = driver.find_elements(By.TAG_NAME, 'tr')

        if not table_rows:
            logging.error(f'Could not locate the table rows for all Orders on DOM. table_Rows: {table_rows}')
            return None, False

        else:
            logging.info(f'Table rows elements found:\n {table_rows}\nTotal number of orders: {len(table_rows)}\n')
            # Exclude header row elem; inclusive from 1st idx

            logging.info(f"First element before slicing: {table_rows[0].text}")
            table_rows_ = table_rows[1:]
            logging.info(f"First element after slicing: {table_rows[0].text}")


            # logging.info(f'***************** table_rows: {len(table_rows)}')
            return table_rows_, True # this returns table_rows with length of 16
        
    except Exception as e:
        print(e)
        
table_rows, table_founds_found = get_all_table_rows_except_header_row()

len(table_rows)

12

In [124]:
table_rows

[<selenium.webdriver.remote.webelement.WebElement (session="6b82a0979fac8c9dcc3350794067bb51", element="9320BCAD1198FFA49C18106268ECBFC5_element_92")>,
 <selenium.webdriver.remote.webelement.WebElement (session="6b82a0979fac8c9dcc3350794067bb51", element="9320BCAD1198FFA49C18106268ECBFC5_element_93")>,
 <selenium.webdriver.remote.webelement.WebElement (session="6b82a0979fac8c9dcc3350794067bb51", element="9320BCAD1198FFA49C18106268ECBFC5_element_94")>,
 <selenium.webdriver.remote.webelement.WebElement (session="6b82a0979fac8c9dcc3350794067bb51", element="9320BCAD1198FFA49C18106268ECBFC5_element_95")>,
 <selenium.webdriver.remote.webelement.WebElement (session="6b82a0979fac8c9dcc3350794067bb51", element="9320BCAD1198FFA49C18106268ECBFC5_element_96")>,
 <selenium.webdriver.remote.webelement.WebElement (session="6b82a0979fac8c9dcc3350794067bb51", element="9320BCAD1198FFA49C18106268ECBFC5_element_97")>,
 <selenium.webdriver.remote.webelement.WebElement (session="6b82a0979fac8c9dcc3350794067

In [125]:
# def get_all_table_rows_except_header_row():
#     table_rows = driver.find_elements(By.TAG_NAME, 'tr')
#     
#     return table_rows[1:]
# 
# table_rows = get_all_table_rows_except_header_row()
# len(table_rows)

In [126]:
def wait_for_element(locator, locator_type, timeout):
    try:
        WebDriverWait(driver, timeout).until(
            EC.visibility_of_element_located((locator_type, locator)))
        return True
    except Exception as e:
        logging.exception(f'An unexpected error occurred: {e}')
        return False

In [127]:
def wait_for_and_find_element(locator, locator_type, timeout):
    try:
        wait = wait_for_element(locator, locator_type, timeout)
        if not wait:
            logging.error(f'Tried to wait to locate element via locator "{locator}", but timed out')
            return False, None
        element = driver.find_element(locator_type, locator)
        if not element:
            logging.error(f'Could not find element via locator "{locator}"')
            return True, None
        if wait and element:
            logging.info(f'Found and located element via locator "{locator}"')
            return True, element

    except Exception as NoSuchElementException:
        logging.exception(f'An unexpected error occurred: {NoSuchElementException}')


In [128]:
def find_element_and_click(locator ,locator_type=By.CSS_SELECTOR):
    """
    Finds element and clicks it using `WebElement.click()`
    :param locator:
    :param locator_type:
    :return: Tuple(bool, WebElement)
    """
    try:
        element = driver.find_element(locator_type, locator)
        if not element:
            logging.error(f'Could not locate element via locator "{locator}".')
        element.click()
        return True, element
    except NoSuchElementException:
        logging.exception(f'Element {locator} was not found.')
        return False, None
    except Exception as e:
        logging.exception(f'Error occurred when trying to find and click element with locator: "{locator}" resulting in error message: {str(e)}')
        return False, None


In [129]:
def find_and_click_exit_button():
    exit_btn_clicked, _ = find_element_and_click(
        locator='//*[@id="MerchantApp"]/div/div/div[3]/div[2]/div[2]/div/div/div[1]/nav/div[1]/div[1]/div/button',
        locator_type=By.XPATH)
    return exit_btn_clicked


In [130]:
def process_row(table_row, order_counter):
    table_row.click()
    time.sleep(5)  # wait for ssb to load on dom
    
    found, elem = wait_for_and_find_element(locator="//*[@class='styles__SidesheetContent-sc-czzuxh-2 hKVVOI']", locator_type=By.XPATH, timeout=10)
    
    if not found or not elem:
        logging.error(f'Could not extract order data in process_row for Order #: {order_counter}')
        return None, order_counter
    
    current_order = elem.text
    if not current_order:
        logging.error(f'Could not get extract data from current order via element.text')
        return None, order_counter

    logging.info(f'Extracted order data for Order #: {order_counter}')

    exit_btn_clicked = find_and_click_exit_button()

    if exit_btn_clicked:
        logging.info(f'Exiting sidesheet body for Order #: {order_counter}')
        return current_order, order_counter + 1
    else:
        logging.error(f'Could not click the exit button for Order #: {order_counter}')
        return None, order_counter


In [131]:
# logging.info(f'table_rows\n: {table_rows}\n')
# print(table_rows)
# len(table_rows)

In [132]:
# def iterate_table_rows(table_rows):
#     orders = []
#     idx = 0
#     while idx < len(table_rows):
#         table_row = table_rows[idx]
# 
#         new_order = process_row(table_row)
#         if new_order is not None:
#             orders.append(new_order)
#             idx += 1
#         else:
#             logging.error(f"Failed to process Order #: {idx + 1}")
#     return orders


In [133]:
def iterate_table_rows(table_rows):
    orders = []
    order_counter = 1
    for table_row in table_rows:
        current_order, new_order_counter = process_row(table_row, order_counter)
        if current_order:
            orders.append(current_order)
        order_counter = new_order_counter
    return orders


In [134]:
orders = iterate_table_rows(table_rows)

In [136]:
orders

['Order: A348DFB6\nDelivered\nThe order was delivered at 11:47 PM on September 28, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nMelissa C\nDasher\nIrving\nPicked Up\n11:30 PM\n(Quoted: 11:14 PM)\nSeptember 28, 2023\nDropped Off\n11:47 PM\n(Quoted: 11:34 PM)\nSeptember 28, 2023\nPick Up Location\n801 Lafayette St, Laredo, TX 78041, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n1\n×\nIce Small (Fountain, Frozen, & Tea)\n$4.99\n2\n×\nTXB Water Bottle (24 pk) (16.9 oz) (Water TX)\n$14.98\nSubtotal\n$19.97\nSubtotal Tax\n$0.41\nCommission (23%)\n-$4.59\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$15.79\nAssociated Transactions (1)\nTransaction #8087670804 - Delivery\n$15.79',
 'Order: EC218FA4\nDelivered\nThe order was delivered at 11:22 PM on September 28, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nPolo M\nDasher\nLeticia\nPicked Up\n11:16 PM\n(Quoted: 11:10 PM)\nSeptember 28, 2023\nDropped Off\n11:22 PM\n(Quoted: 11:24 PM)\nSept

In [137]:
len(orders)

12

In [145]:
def orders_scraper():
    table_rows, table_rows_present = get_all_table_rows_except_header_row()

    if not table_rows and not table_rows_present:
        logging.error(f'Could not find table_rows: {table_rows}| table_rows_present: {table_rows_present}')

    orders = iterate_table_rows(table_rows)

    if not orders:
        logging.error(f'Could get orders')

    if table_rows and table_rows_present and orders:
        logging.info(f'Successfully scraped all orders in the table. ')

    logging.info(f'\n***********************************\n {orders} \n********************************\n')

    return orders


In [146]:
def scrape_orders_data():
    try:
        scraped_orders_data = orders_scraper()
        if not scraped_orders_data:
            logging.error('Could not orders_page.scrape_orders_table_data')
            return None
        else:
            logging.info('Successfully orders_page.scrape_orders_data')
            return scraped_orders_data
    except Exception as e:
        logging.exception(f'An error occurred trying to orders_page.scrape_orders_table_data: {e}')
        return None



In [147]:
data = scrape_orders_data()
data

['Order: A348DFB6\nDelivered\nThe order was delivered at 11:47 PM on September 28, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nMelissa C\nDasher\nIrving\nPicked Up\n11:30 PM\n(Quoted: 11:14 PM)\nSeptember 28, 2023\nDropped Off\n11:47 PM\n(Quoted: 11:34 PM)\nSeptember 28, 2023\nPick Up Location\n801 Lafayette St, Laredo, TX 78041, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n1\n×\nIce Small (Fountain, Frozen, & Tea)\n$4.99\n2\n×\nTXB Water Bottle (24 pk) (16.9 oz) (Water TX)\n$14.98\nSubtotal\n$19.97\nSubtotal Tax\n$0.41\nCommission (23%)\n-$4.59\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$15.79\nAssociated Transactions (1)\nTransaction #8087670804 - Delivery\n$15.79',
 'Order: EC218FA4\nDelivered\nThe order was delivered at 11:22 PM on September 28, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nPolo M\nDasher\nLeticia\nPicked Up\n11:16 PM\n(Quoted: 11:10 PM)\nSeptember 28, 2023\nDropped Off\n11:22 PM\n(Quoted: 11:24 PM)\nSept

In [101]:
# def orders_scraper():
#     orders = []
# 
#     for idx, table_row in enumerate(table_rows):
#         if idx >= 1:
#             table_row.click()
# 
#             time.sleep(5)  # wait for ssb to load on dom
# 
#             found, elem = wait_for_and_find_element(locator="//*[@class='styles__SidesheetContent-sc-czzuxh-2 hKVVOI']", locator_type=By.XPATH, timeout=10)
# 
# 
#             start_length = len(orders)
#             orders.append(elem.text)
#             end_length = len(orders)
# 
#             if end_length > start_length:
#                 exit_btn_clicked = find_element_and_click(locator='//*[@id="MerchantApp"]/div/div/div[3]/div[2]/div[2]/div/div/div[1]/nav/div[1]/div[1]/div/button', locator_type=By.XPATH)
# 
#                 if exit_btn_clicked:
#                     logging.info(f'Exiting sidesheetbody for Order #: {idx}')
# 
#     return orders


In [102]:
# orders = orders_scraper()
# orders

In [47]:
orders = ['Order: A348DFB6\nDelivered\nThe order was delivered at 11:47 PM on September 28, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nMelissa C\nDasher\nIrving\nPicked Up\n11:30 PM\n(Quoted: 11:14 PM)\nSeptember 28, 2023\nDropped Off\n11:47 PM\n(Quoted: 11:34 PM)\nSeptember 28, 2023\nPick Up Location\n801 Lafayette St, Laredo, TX 78041, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n1\n×\nIce Small (Fountain, Frozen, & Tea)\n$4.99\n2\n×\nTXB Water Bottle (24 pk) (16.9 oz) (Water TX)\n$14.98\nSubtotal\n$19.97\nSubtotal Tax\n$0.41\nCommission (23%)\n-$4.59\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$15.79\nAssociated Transactions (1)\nTransaction #8087670804 - Delivery\n$15.79',
 'Order: EC218FA4\nDelivered\nThe order was delivered at 11:22 PM on September 28, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nPolo M\nDasher\nLeticia\nPicked Up\n11:16 PM\n(Quoted: 11:10 PM)\nSeptember 28, 2023\nDropped Off\n11:22 PM\n(Quoted: 11:24 PM)\nSeptember 28, 2023\nPick Up Location\n800 W Interstate 20, Big Spring, TX 79720, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n1\n×\nIce Small (Fountain, Frozen, & Tea)\n$4.99\nSubtotal\n$4.99\nSubtotal Tax\n$0.41\nCommission (16%)\n-$0.80\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$4.60\nAssociated Transactions (1)\nTransaction #8087614462 - Delivery\n$4.60',
 'Order: 7D285B12\nDelivered\nThe order was delivered at 10:23 PM on September 28, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nAngelina N\nDasher\nTerrah\nPicked Up\n10:09 PM\n(Quoted: 10:12 PM)\nSeptember 28, 2023\nDropped Off\n10:23 PM\n(Quoted: 10:31 PM)\nSeptember 28, 2023\nPick Up Location\n800 W Interstate 20, Big Spring, TX 79720, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n1\n×\nBig Red Bottle (20oz) (Soda TX)\n$3.79\n1\n×\nFountain Drink (Fountain, Frozen, & Tea)\nDrink Selection:\nDr Pepper\n$0.00\n1\n×\nCrush Orange Bottle (20oz) (Soda TX)\n$3.79\n1\n×\nBig Blue Bottle (20oz) (Soda TX)\n$3.79\nSubtotal\n$11.37\nSubtotal Tax\n$0.94\nCommission (23%)\n-$2.62\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$9.69\nAssociated Transactions (1)\nTransaction #8087246804 - Delivery\n$9.69',
 "Order: F9F46AEF\nDelivered\nThe order was delivered at 7:47 PM on September 28, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nMiranda S\nDasher\nPedro\nPicked Up\n7:34 PM\n(Quoted: 7:36 PM)\nSeptember 28, 2023\nDropped Off\n7:47 PM\n(Quoted: 8:06 PM)\nSeptember 28, 2023\nPick Up Location\n1001 W Van Alstyne Pkwy, Van Alstyne, TX 75495, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n1\n×\nCheetos Crunchy Flamin' Hot (3.25 oz) (Chips & Popcorn TX)\n$3.19\n1\n×\nSquirt Bottle (20 oz) (Soda TX)\n$3.19\n2\n×\nDr Pepper Bottle (20 oz) (Soda TX)\n$7.58\n1\n×\nDoritos Nacho Cheese (9.25 oz) (Chips & Popcorn TX)\n$6.99\nSubtotal\n$20.95\nSubtotal Tax\n$0.89\nCommission (23%)\n-$4.82\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$17.02\nAssociated Transactions (1)\nTransaction #8085798910 - Delivery\n$17.02",
 'Order: 7D8C6B67\nDelivered\nThe order was delivered at 6:07 PM on September 28, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nElena R\nDasher\nSophia\nPicked Up\n5:54 PM\n(Quoted: 5:58 PM)\nSeptember 28, 2023\nDropped Off\n6:07 PM\n(Quoted: 6:32 PM)\nSeptember 28, 2023\nPick Up Location\n3702 FM2147, Horseshoe Bay, TX 78657, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n1\n×\nSprite Bottle (20 oz) (Soda TX)\n$3.19\n2\n×\nDr Pepper Bottle (20 oz) (Soda TX)\n$7.58\nSubtotal\n$10.77\nSubtotal Tax\n$0.89\nCommission (20%)\n-$2.15\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$9.51\nAssociated Transactions (1)\nTransaction #8084663509 - Delivery\n$9.51',
 'Order: 702662D6\nDelivered\nThe order was delivered at 4:16 PM on September 28, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nDavid L\nDasher\nLisa\nPicked Up\n3:59 PM\n(Quoted: 4:06 PM)\nSeptember 28, 2023\nDropped Off\n4:16 PM\n(Quoted: 4:25 PM)\nSeptember 28, 2023\nPick Up Location\n201 S 9th St, Durant, OK 74701, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n1\n×\nKeystone Light Beer Can (12 oz x 15 ct) (Domestic Beer OK)\n$16.29\nSubtotal\n$16.29\nSubtotal Tax\n$1.53\nAlcohol Fee\n-$1.00\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$15.29\nAssociated Transactions (1)\nTransaction #8083626907 - Delivery\n$15.29',
 'Order: 00BEC8AB\nDelivered\nThe order was delivered at 11:48 AM on September 28, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nWesley W\nDasher\nAndres\n1 previous Dasher\nPicked Up\n11:45 AM\n(Quoted: 11:38 AM)\nSeptember 28, 2023\nDropped Off\n11:48 AM\n(Quoted: 11:48 AM)\nSeptember 28, 2023\nPick Up Location\n816 N Center St, Bonham, TX 75418, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n1\n×\nHand-Breaded Chicken Roll Sandwich (Hot Sandwiches TXB#6)\nDipping Sauce:\nHoney Mustard Dipping Sauce\nOriginal or Spicy:\nOriginal\n$4.19\n1\n×\nFountain Drink (Fountain, Frozen, & Tea)\nDrink Selection:\nDiet Dr Pepper\n$0.00\nSubtotal\n$4.19\nSubtotal Tax\n$0.35\nCommission (16%)\n-$0.67\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$3.87\nAssociated Transactions (1)\nTransaction #8081758649 - Delivery\n$3.87',
 "Order: F20F6AE7\nDelivered\nThe order was delivered at 2:55 AM on September 28, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nMisty D\nDasher\nMartin\nPicked Up\n2:42 AM\n(Quoted: 2:32 AM)\nSeptember 28, 2023\nDropped Off\n2:55 AM\n(Quoted: 2:53 AM)\nSeptember 28, 2023\nPick Up Location\n801 Lafayette St, Laredo, TX 78041, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n1\n×\nCheetos Crunchy Flamin' Hot (3.25 oz) (Chips & Popcorn TX)\n$3.19\n1\n×\nBig Red Can (12 pk-12 oz) (Soda TX)\n$11.29\n1\n×\nIce Small (Fountain, Frozen, & Tea)\n$4.99\nSubtotal\n$19.47\nSubtotal Tax\n$1.34\nCommission (16%)\n-$3.12\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$17.69\nAssociated Transactions (1)\nTransaction #8080315650 - Delivery\n$17.69",
 "Order: CA9E35CB\nDelivered\nThe order was delivered at 2:28 AM on September 28, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nAntonio G\nDasher\nRogelio\nPicked Up\n2:02 AM\n(Quoted: 1:59 AM)\nSeptember 28, 2023\nDropped Off\n2:28 AM\n(Quoted: 2:33 AM)\nSeptember 28, 2023\nPick Up Location\n801 Lafayette St, Laredo, TX 78041, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n1\n×\nKettle Chips Jalapeno (5oz) (Chips & Popcorn TX)\n$6.29\n1\n×\nA&W Caffeine-Free Root Beer Bottle (20 oz) (Soda TX)\n$3.79\n1\n×\nCheetos Crunchy Flamin' Hot Limon (8.5 oz) (Chips & Popcorn TX)\n$6.49\nSubtotal\n$16.57\nSubtotal Tax\n$0.31\nCommission (16%)\n-$2.65\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$14.23\nAssociated Transactions (1)\nTransaction #8080266865 - Delivery\n$14.23",
 "Order: D5782A52\nCancelled - Paid\nThe order was cancelled on September 28, 2023 at 1:00 AM. You’re paid for cancelled orders when you’ve successfully confirmed an order, it was prepared, and you did not initiate or are not at fault for the cancellation. Learn More\nRate Dasher\nGet help\nDelivery Details\nCustomer\njenna s\nDasher\nTo be assigned\n1 previous Dasher\nEstimated Pick Up\n12:44 AM\nSeptember 28, 2023\nEstimated Drop Off\n1:11 AM\nSeptember 28, 2023\nPick Up Location\n2503 TX-349, Midland, TX 79706, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n1\n×\nHershey's Plain King Size (2.6 oz) (Chocolate TX)\n$3.79\n1\n×\nKraft Philadelphia Cream Cheese, 8oz (Cheese TX)\n$8.39\nSubtotal\n$12.18\nSubtotal Tax\n$0.30\nCommission (23%)\n-$2.80\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$9.68\nAssociated Transactions (1)\nTransaction #8080161083 - Delivery\n$9.68",
 'Order: 5CDF188C\nCancelled - Paid\nThe order was cancelled on September 28, 2023 at 12:45 AM. You’re paid for cancelled orders when you’ve successfully confirmed an order, it was prepared, and you did not initiate or are not at fault for the cancellation. Learn More\nGet help\nDelivery Details\nCustomer\nErin B\nDasher\nTo be assigned\nEstimated Pick Up\n12:02 AM\nSeptember 28, 2023\nEstimated Drop Off\n12:31 AM\nSeptember 28, 2023\nPick Up Location\n3702 FM2147, Horseshoe Bay, TX 78657, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n4\n×\nCorona Extra Beer Can (24 oz) (Import Beer 53/66/67)\n$22.76\nSubtotal\n$22.76\nSubtotal Tax\n$1.88\nCommission (25%)\n-$5.69\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$18.95\nAssociated Transactions (1)\nTransaction #8080129920 - Delivery\n$18.95',
 "Order: 0BC3FC5E\nDelivered\nThe order was delivered at 12:36 AM on September 28, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nReice W\nDasher\nCynthia\nPicked Up\n12:25 AM\n(Quoted: 12:13 AM)\nSeptember 28, 2023\nDropped Off\n12:36 AM\n(Quoted: 12:37 AM)\nSeptember 28, 2023\nPick Up Location\n2320 W Main St, Durant, OK 74701, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n1\n×\nLay's Classic Potato Chips (7.75 oz) (Chips & Popcorn OK)\n$5.79\n1\n×\nReese's Peanut Butter Cups King Size (2.84 oz) (Chocolate OK)\n$3.39\n1\n×\nAir Heads Bites (2oz) (Fruity OK)\n$4.99\n1\n×\nCrush Orange Can (12pk 12oz) (Soda OK)\n$11.29\nSubtotal\n$25.46\nPromo Paid by DoorDash\n-$15.00\n(60% off, up to $15)\nSubtotal for Tax Calculations\n$10.46\nSubtotal Tax\n$0.98\nCommission (23%)\n-$5.86\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$19.60\nAssociated Transactions (1)\nTransaction #8080078554 - Delivery\n$19.60"]

In [48]:
def clean_order_text(order):
    # Define the patterns in a list
    patterns = [
        re.compile(r'(Rate Dasher|Learn More|Get help).*?(?=Pick Up Location)', re.DOTALL),
        re.compile(r'Channel.*?(?=Order Details)', re.DOTALL),
        re.compile(r'Associated Transactions \(\d+\).*?(?=Transaction #\d+ - Delivery)', re.DOTALL),
        re.compile(r'Associated Transactions \(\d+\).*?(?=Transaction #\d+ - Pickup)', re.DOTALL)

    ]

    # Apply each pattern
    for pattern in patterns:
        order = re.sub(pattern, '', order).strip()

    # Replace newline characters with spaces
    order = order.replace('\n', ' ')
    print(f'order **************: {order}')

    return order


In [49]:
def parse_menu_items(price_as_value):
    # Initialize a dictionary for the nested order details
    item_name_to_item_price = {}

    # Regular expression pattern to match an item and its corresponding price
    pattern = re.compile(r'(.+?) (\$\d+\.\d+)')

    # Find all matches in the string
    matches = pattern.findall(price_as_value)

    # Loop through all the matches and populate the dictionary
    for item_name, item_price in matches:
        item_name_to_item_price[item_name] = item_price

    return item_name_to_item_price


In [50]:
def parse_order(order):
    # Define the known keys with their regex patterns
    known_keys = [r'\bOrder: \b', r'\bDelivered\b', r'\bCustomer\sPicked\sUp\b', r'\bCancelled\s-\sNot\sPaid\b', r'\bCancelled\s-\sPaid\b', r'\bPick Up Location\b', r'\bOrder Details\b',
                  r'\bSubtotal\b(?=\s[^a-zA-Z])', r'\bSubtotal\sTax\b', r'Commission \(\d+%\)',
                  r'\bTotal Customer Refund\b', r'\bEstimated Payout\b', r'\bAssociated Transactions \(\d+%\)',
                  r'Transaction #\d+ - Delivery']

    # Initialize the dictionary to store our parsed values
    order_dict = {}

    # Find positions of known keys
    positions = []
    for key in known_keys:
        for m in re.finditer(key, order):
            positions.append((m.start(), m.end(), m.group()))

    # Sort positions by their start index
    positions.sort(key=lambda x: x[0])
    # print(positions)

    # Create segments based on positions
    segments = []

    for i in range(len(positions)):
        start = positions[i][1]
        end = positions[i + 1][0] if i + 1 < len(positions) else len(order)
        key = positions[i][2]
        value = order[start:end].strip().split(' ')[0] if 'Estimated Payout' in key else order[start:end].strip()
        segments.append((key, value))

    # Parse segments into dictionary
    for key, value in segments:
        if key == "Order Details":
            value = parse_menu_items(value)
        order_dict[key.replace(':', '').strip()] = value

    return order_dict


In [51]:
def main_looper_func(orders):

    results = []

    for order in orders:
        # 1. Get order strings squeaky clean!
        order_cleaned = clean_order_text(order)

        # 2. Parse each cleaned order string into formatted key/value pairs
        parsed_order = parse_order(order_cleaned)

        results.append(parsed_order)

    return results


In [52]:
import re
import json

In [53]:
results = main_looper_func(orders)

order **************: Order: A348DFB6 Delivered The order was delivered at 11:47 PM on September 28, 2023.  Pick Up Location 801 Lafayette St, Laredo, TX 78041, USA Order Details 1 × Ice Small (Fountain, Frozen, & Tea) $4.99 2 × TXB Water Bottle (24 pk) (16.9 oz) (Water TX) $14.98 Subtotal $19.97 Subtotal Tax $0.41 Commission (23%) -$4.59 Total Customer Refund -$0.00 Estimated Payout $15.79 Transaction #8087670804 - Delivery $15.79
order **************: Order: EC218FA4 Delivered The order was delivered at 11:22 PM on September 28, 2023.  Pick Up Location 800 W Interstate 20, Big Spring, TX 79720, USA Order Details 1 × Ice Small (Fountain, Frozen, & Tea) $4.99 Subtotal $4.99 Subtotal Tax $0.41 Commission (16%) -$0.80 Total Customer Refund -$0.00 Estimated Payout $4.60 Transaction #8087614462 - Delivery $4.60
order **************: Order: 7D285B12 Delivered The order was delivered at 10:23 PM on September 28, 2023.  Pick Up Location 800 W Interstate 20, Big Spring, TX 79720, USA Order Det

In [54]:
def prettify_list_of_dicts(list_of_dicts):
    return json.dumps(list_of_dicts, indent=4)


print(prettify_list_of_dicts(results))



[
    {
        "Order": "A348DFB6",
        "Delivered": "The order was delivered at 11:47 PM on September 28, 2023.",
        "Pick Up Location": "801 Lafayette St, Laredo, TX 78041, USA",
        "Order Details": {
            "1 \u00d7 Ice Small (Fountain, Frozen, & Tea)": "$4.99",
            " 2 \u00d7 TXB Water Bottle (24 pk) (16.9 oz) (Water TX)": "$14.98"
        },
        "Subtotal": "$19.97",
        "Subtotal Tax": "$0.41",
        "Commission (23%)": "-$4.59",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$15.79",
        "Transaction #8087670804 - Delivery": "$15.79"
    },
    {
        "Order": "EC218FA4",
        "Delivered": "The order was delivered at 11:22 PM on September 28, 2023.",
        "Pick Up Location": "800 W Interstate 20, Big Spring, TX 79720, USA",
        "Order Details": {
            "1 \u00d7 Ice Small (Fountain, Frozen, & Tea)": "$4.99"
        },
        "Subtotal": "$4.99",
        "Subtotal Tax": "$0.41",
        "Commi

In [1]:
list_of_mapped_orders = [{'Order': '42C2A525', 'Delivered': 'The order was delivered at 11:41 PM on October 1, 2023.', 'Pick Up Location': '5004 Wesley St, Greenville, TX 75402, USA', 'Order Details': {'1 × Coca-Cola Can (12 pk-12 oz) (Soda TX)': '$11.29', ' 1 × TXB Water Bottle (24 pk) (16.9 oz) (Water TX)': '$7.49', ' 1 × Coke Classic Bottle (20oz) (Soda TX)': '$3.19'}, 'Subtotal': '$21.97', 'Subtotal Tax': '$1.19', 'Commission (23%)': '-$5.05', 'Total Customer Refund': '-$0.00', 'Estimated Payout': '$18.11', 'Transaction #8113346069 - Delivery': '$18.11', 'Store Number': '20'}, {'Order': 'AA85B890', 'Delivered': 'The order was delivered at 10:08 PM on October 1, 2023.', 'Pick Up Location': '5004 Wesley St, Greenville, TX 75402, USA', 'Order Details': {'1 × Moon Pie Double Decker Banana (2.75oz) (Pastry TX)': '$2.29', ' 1 × Moon Pie Double Decker Chocolate (2.75oz) (Pastry TX)': '$2.29', ' 1 × TXB Sweet Tea Bottle (16oz) (Tea TX)': '$3.29', ' 1 × TXB Strawberry White Tea Bottle (16oz) (Tea TX)': '$2.59', ' 1 × TXB Premium Electrolyte Water Bottle (12pk 1L) (Water TX)': '$4.09'}, 'Subtotal': '$14.55', 'Subtotal Tax': '$0.59', 'Commission (16%)': '-$2.33', 'Total Customer Refund': '-$0.00', 'Estimated Payout': '$12.81', 'Transaction #8112864098 - Delivery': '$12.81', 'Store Number': '20'}, {'Order': '773C7330', 'Delivered': 'The order was delivered at 9:07 PM on October 1, 2023.', 'Pick Up Location': '5004 Wesley St, Greenville, TX 75402, USA', 'Order Details': {'1 × Lone Star Beer Can (16 oz x 6 ct) (Domestic Beer NT)': '$13.79'}, 'Subtotal': '$13.79', 'Subtotal Tax': '$1.14', 'Commission (16%)': '-$2.21', 'Total Customer Refund': '-$0.00', 'Estimated Payout': '$12.72', 'Transaction #8112394394 - Delivery': '$12.72', 'Store Number': '20'}, {'Order': 'EDCB46B1', 'Delivered': 'Error Charge The customer reported one or more missing items resulting in an error charge of $5.86. If you believe this was an error, you can dispute the error charge by October 15, 2023.', 'Pick Up Location': '1001 W Van Alstyne Pkwy, Van Alstyne, TX 75495, USA', 'Order Details': {'1 × Michelob Ultra Light Beer Can (12 oz x 12 ct) (Domestic Beer NT)': '$27.49', ' 1 × Truly Hard Seltzer Berry Mix Pack Can (12 oz x 12 ct) (Seltzer NT)': '$28.79'}, 'Subtotal': '-$5.30 Error Charge Tax -$0.56 Error Charge Total -$5.86 Order Total $47.98', 'Commission (23%)': '-$12.94 Order Total $47.98 Error Charges 1 × Michelob Ultra Light Beer Can (12 oz x 12 ct) (Domestic Beer NT) Missing Item -$5.30 Error Charge', 'Estimated Payout': '$42.12', 'Transaction #8109695187 - Delivery': '$47.98 Transaction #8109811800 - Error Charges -$5.86', 'Store Number': '32'}, {'Order': '68944F97', 'Delivered': 'The order was delivered at 2:50 PM on October 1, 2023.', 'Pick Up Location': '800 W Interstate 20, Big Spring, TX 79720, USA', 'Order Details': {'2 × Michelob Ultra Light Beer Can (12 oz x 18 ct) (Domestic Beer ST)': '$72.58'}, 'Subtotal': '$72.58', 'Subtotal Tax': '$5.99', 'Commission (23%)': '-$16.69', 'Total Customer Refund': '-$0.00', 'Estimated Payout': '$61.88', 'Transaction #8108722869 - Delivery': '$61.88', 'Store Number': '70'}, {'Order': 'ED39CFE2', 'Delivered': 'Error Charge The customer reported one or more missing items resulting in an error charge of $8.11. If you believe this was an error, you can dispute the error charge by October 15, 2023.', 'Pick Up Location': '1301 N U.S. Hwy 281, Marble Falls, TX 78654, USA', 'Order Details': {'1 × Ice Large (Fountain, Frozen, & Tea)': '$7.49', ' 1 × Campbells Chicken Noodle Bowl (15.4oz) (Pantry TX)': '$5.29', ' 2 × TXB Premium Electrolyte Water (1 lt) (Water TX)': '$8.78', ' 1 × 7 Up Bottle (20 oz) (Soda TX)': '$3.79', ' 2 × Gatorade Cool Blue (28 oz) (Sports Drinks TX)': '$9.58'}, 'Subtotal': '-$7.50 Error Charge Tax -$0.61 Error Charge Total -$8.11 Order Total $28.36', 'Commission (25%)': '-$8.73 Order Total $28.36 Error Charges 1 × Ice Large (Fountain, Frozen, & Tea) Missing Item -$7.50 Error Charge', 'Estimated Payout': '$20.25', 'Transaction #8107801352 - Delivery': '$28.36 Transaction #8107871093 - Error Charges -$8.11', 'Store Number': '53'}, {'Order': '78BAE2F8', 'Delivered': 'The order was delivered at 12:41 PM on October 1, 2023.', 'Pick Up Location': '1001 S State Hwy 16, Fredericksburg, TX 78624, USA', 'Order Details': {'1 × Crush Grape Bottle (20oz) (Soda TX)': '$3.79', ' 1 × Turkey & Bacon Wrap (Salads, Wraps, & Cold Sandwiches TXB#61)': '$8.79', ' Subtotal for Tax Calculations': '$12.58'}, 'Subtotal Tax': '$1.04', 'Commission (16%)': '-$2.01', 'Total Customer Refund': '-$0.00', 'Estimated Payout': '$11.61', 'Transaction #8107649354 - Delivery': '$11.61', 'Store Number': '61'}, {'Order': 'DE4528F2', 'Delivered': 'The order was delivered at 12:39 PM on October 1, 2023.', 'Pick Up Location': '2320 W Main St, Durant, OK 74701, USA', 'Order Details': {'1 × Coke Classic Bottle (20oz) (Soda OK)': '$3.19', ' 2 × Red Bull Energy Drink Can (12 oz) (Energy Drinks OK)': '$11.58'}, 'Subtotal': '$14.77', 'Subtotal Tax': '$1.38', 'Commission (23%)': '-$3.40', 'Total Customer Refund': '-$0.00', 'Estimated Payout': '$11.37', 'Transaction #8107692197 - Delivery': '$11.37', 'Store Number': '16'}, {'Order': 'F5539B37', 'Delivered': 'The order was delivered at 10:52 AM on October 1, 2023.', 'Pick Up Location': '3702 FM2147, Horseshoe Bay, TX 78657, USA', 'Order Details': {'2 × Chorizo Egg Cheese Quesadilla (Breakfast Tacos & Quesadillas) Salsa: Salsa Roja': '$14.98'}, 'Subtotal': '$14.98', 'Subtotal Tax': '$1.24', 'Commission (25%)': '-$3.75', 'Total Customer Refund': '-$0.00', 'Estimated Payout': '$12.47', 'Transaction #8106904074 - Delivery': '$12.47', 'Store Number': '67'}, {'Order': 'FE329688', 'Delivered': 'The order was delivered at 10:03 AM on October 1, 2023.', 'Pick Up Location': '800 W Interstate 20, Big Spring, TX 79720, USA', 'Order Details': {'3 × Red Bull Energy Drink Can (12 oz) (Energy Drinks TX)': '$17.37'}, 'Subtotal': '$17.37', 'Subtotal Tax': '$1.43', 'Commission (16%)': '-$2.78', 'Total Customer Refund': '-$0.00', 'Estimated Payout': '$16.02', 'Transaction #8106720721 - Delivery': '$16.02', 'Store Number': '70'}, {'Order': 'C5A903F0', 'Cancelled - Paid': 'The order was cancelled on October 1, 2023 at 7:31 AM. You’re paid for cancelled orders when you’ve successfully confirmed an order, it was prepared, and you did not initiate or are not at fault for the cancellation.', 'Pick Up Location': '2503 TX-349, Midland, TX 79706, USA', 'Order Details': {'1 × Sausage Egg Cheese Taco (Breakfast Tacos & Quesadillas)': '$3.79'}, 'Subtotal': '$3.79', 'Subtotal Tax': '$0.30', 'Commission (16%)': '-$0.61', 'Total Customer Refund': '-$0.00', 'Estimated Payout': '$3.48', 'Transaction #8106298092 - Delivery': '$3.48', 'Store Number': '71'}, {'Order': '782881C7', 'Delivered': 'The order was delivered at 3:08 AM on October 1, 2023.', 'Pick Up Location': '145 Lehman Rd, Kyle, TX 78640, USA', 'Order Details': {'1 × Hostess Powdered Donettes Bag (10.5oz) (Donuts TX)': '$6.39', ' 1 × Fountain Drink (Fountain, Frozen, & Tea) Drink Selection: Sprite': '$0.00', ' 1 × Fountain Drink (Fountain, Frozen, & Tea) Drink Selection: TXB Cherry Limeade': '$0.00', ' 1 × Fountain Drink (Fountain, Frozen, & Tea) Drink Selection: TXB Root Beer': '$0.00'}, 'Subtotal': '$6.39', 'Subtotal Tax': '$0.00', 'Commission (16%)': '-$1.02', 'Total Customer Refund': '-$0.00 Refunds $6.99', 'Estimated Payout': '$12.36', 'Transaction #8105816170 - Delivery': '$5.37', 'Store Number': '68'}, {'Order': 'ECCAB249', 'Delivered': 'The order was delivered at 2:32 AM on October 1, 2023.', 'Pick Up Location': '1402 Williams Dr, Georgetown, TX 78628, USA', 'Order Details': {'1 × Chicken Tender Meal Kit (Meal Kits TXB#65)': '$9.19'}, 'Subtotal': '$9.19', 'Subtotal Tax': '$0.76', 'Commission (16%)': '-$1.47', 'Total Customer Refund': '-$0.00', 'Estimated Payout': '$8.48', 'Transaction #8105817488 - Delivery': '$8.48', 'Store Number': '65'}, {'Order': '8E90EF84', 'Delivered': 'The order was delivered at 1:24 AM on October 1, 2023.', 'Pick Up Location': '21024 W, 21024 State Hwy 71, Spicewood, TX 78669, USA', 'Order Details': {'1 × Pork Tamale Meal Kit (Meal Kits TXB#62)': '$9.19'}, 'Subtotal': '$9.19', 'Subtotal Tax': '$0.76', 'Commission (16%)': '-$1.47', 'Total Customer Refund': '-$0.00', 'Estimated Payout': '$8.48', 'Transaction #8105638611 - Delivery': '$8.48', 'Store Number': '62'}, {'Order': 'B944ED77', 'Cancelled - Not Paid': 'The order was cancelled on October 1, 2023 at 12:57 AM. You were not paid because you were unable to fulfill the order.', 'Pick Up Location': '1402 Williams Dr, Georgetown, TX 78628, USA', 'Order Details': {'2 × Black Box Pinot Grigio Tetra 500ml Box (12% ABV) (Specialty Wine)': '$12.58'}, 'Subtotal': '$12.58', 'Subtotal Tax': '$1.04', 'Commission (16%)': '-$2.01', 'Total Customer Refund': '-$0.00', 'Estimated Payout': '$0.00', 'Store Number': '65'}, {'Order': 'E7A89C68', 'Delivered': 'The order was delivered at 12:43 AM on October 1, 2023.', 'Pick Up Location': '1402 Williams Dr, Georgetown, TX 78628, USA', 'Order Details': {'1 × Pepcid AC Maximum Strength Tablet (8ct) (Medicine TX)': '$14.39', ' 1 × 2% Milk (16 oz) (Milk TX)': '$3.79'}, 'Subtotal': '$18.18', 'Subtotal Tax': '$0.00', 'Commission (16%)': '-$2.91', 'Total Customer Refund': '-$0.00', 'Estimated Payout': '$15.27', 'Transaction #8105526412 - Delivery': '$15.27', 'Store Number': '65'}, {'Order': 'C41B4C30', 'Delivered': 'The order was delivered at 12:43 AM on October 1, 2023.', 'Pick Up Location': '21024 W, 21024 State Hwy 71, Spicewood, TX 78669, USA', 'Order Details': {'1 × Triple Chocolate Chunk Cookie (Pastries & Desserts)': '$2.49', ' 1 × Chocolate Milk (16 oz) (Milk TX)': '$3.79', ' 3 × Fountain Drink (Fountain, Frozen, & Tea) Drink Selection: Dr Pepper': '$0.00', ' 1 × Dr Pepper Bottle (20 oz) (Soda TX)': '$3.79'}, 'Subtotal': '$10.07', 'Subtotal Tax': '$0.83', 'Commission (16%)': '-$1.61', 'Total Customer Refund': '-$0.00', 'Estimated Payout': '$9.29', 'Transaction #8105477186 - Delivery': '$9.29', 'Store Number': '62'}]


In [2]:
def get_flatten_order(d, parent_key='', sep='.'):
    order = {}
    for k, v in d.items():
        new_key = f'{parent_key}{sep}{k}' if parent_key else k
        if isinstance(v, dict):
            order.update(get_flatten_order(v, new_key, sep=sep))
        else:
            order[new_key] = v
    return order

In [3]:
def get_flattened_orders(orders):
    flattened_orders = []
    for order in orders:
        flattened_order = get_flatten_order(order)
        flattened_orders.append(flattened_order)
    return flattened_orders

In [4]:
flattened_orders = get_flattened_orders(list_of_mapped_orders)
flattened_orders

[{'Order': '42C2A525',
  'Delivered': 'The order was delivered at 11:41 PM on October 1, 2023.',
  'Pick Up Location': '5004 Wesley St, Greenville, TX 75402, USA',
  'Order Details.1 × Coca-Cola Can (12 pk-12 oz) (Soda TX)': '$11.29',
  'Order Details. 1 × TXB Water Bottle (24 pk) (16.9 oz) (Water TX)': '$7.49',
  'Order Details. 1 × Coke Classic Bottle (20oz) (Soda TX)': '$3.19',
  'Subtotal': '$21.97',
  'Subtotal Tax': '$1.19',
  'Commission (23%)': '-$5.05',
  'Total Customer Refund': '-$0.00',
  'Estimated Payout': '$18.11',
  'Transaction #8113346069 - Delivery': '$18.11',
  'Store Number': '20'},
 {'Order': 'AA85B890',
  'Delivered': 'The order was delivered at 10:08 PM on October 1, 2023.',
  'Pick Up Location': '5004 Wesley St, Greenville, TX 75402, USA',
  'Order Details.1 × Moon Pie Double Decker Banana (2.75oz) (Pastry TX)': '$2.29',
  'Order Details. 1 × Moon Pie Double Decker Chocolate (2.75oz) (Pastry TX)': '$2.29',
  'Order Details. 1 × TXB Sweet Tea Bottle (16oz) (Tea 

In [8]:
def convert_orders_to_dataframes(orders):
    dfs = []
    for order in orders:
        s = pd.Series(order)
        df = pd.DataFrame(s)
        dfs.append(df)
        
    
    return dfs
    


In [9]:
dfs = convert_orders_to_dataframes(flattened_orders)
dfs

[                                                                                                    0
 Order                                                                                        42C2A525
 Delivered                                           The order was delivered at 11:41 PM on October...
 Pick Up Location                                            5004 Wesley St, Greenville, TX 75402, USA
 Order Details.1 × Coca-Cola Can (12 pk-12 oz) (...                                             $11.29
 Order Details. 1 × TXB Water Bottle (24 pk) (16...                                              $7.49
 Order Details. 1 × Coke Classic Bottle (20oz) (...                                              $3.19
 Subtotal                                                                                       $21.97
 Subtotal Tax                                                                                    $1.19
 Commission (23%)                                                        

In [78]:
def get_store_num_from_store_addrs():
    # todo
    # pass
    return '#101'

In [15]:
from datetime import datetime

def get_excel_file_name():
    
    today = datetime.today().strftime('%m.%d.%y')
    output_file_name = f'DD {today}.xlsx'
    return output_file_name

excel_file_name = get_excel_file_name()
excel_file_name

'DD 10.03.23.xlsx'

In [18]:
def get_sheet_name(idx):
    return f"Sheet_{idx}"


In [22]:
def set_format(worksheet, start_col, end_col, cell_format):
    """Set format for a range of columns."""
    worksheet.set_column(start_col, end_col, None, cell_format)

def find_max_lengths(df):
    """Find the maximum string lengths for each column in a DataFrame."""
    max_lengths = {}
    for col in df.columns:
        max_lengths[col] = max(df[col].astype(str).apply(len).max(),
                                len(str(col)))
    return max_lengths

def apply_column_formats(worksheet, max_lengths, cell_format):
    print("Applying column formats...") 
    
    """Apply column formatting and widths based on content length."""
    for idx, col in enumerate(max_lengths.keys()):
        col_width = max_lengths[col] + 1  # Add 1 for a bit of padding
        set_format(worksheet, idx + 1, idx + 1, cell_format)
        worksheet.set_column(idx + 1, idx + 1, col_width)

def apply_first_column_format(worksheet, df, cell_format):
    print("Applying first column format...") 
    """Apply formatting to the first column and adjust its width."""
    index_as_series = pd.Series(df.index.astype(str))
    index_length = max(index_as_series.apply(len).max(),
                       len(str(df.index.name)))
    col_width = index_length + 1
    worksheet.set_column(0, 0, col_width, cell_format)


def apply_sheet_formats(xlsxwriter, sheet_name, df):
    """Apply sheet formats."""
    workbook = xlsxwriter.book
    worksheet = xlsxwriter.sheets[sheet_name]

    # Create text wrap format with large font, aligned to center
    wrap_txt_format = workbook.add_format({
        'text_wrap': True,
        'align': 'center',
        'valign': 'vcenter',
        'font_size': 36,
    })

    # Create bold wrap format for the first column
    bold_wrap_format = workbook.add_format({
        'text_wrap': True,
        'bold': True,
        'align': 'center',
        'valign': 'vcenter',
        'font_size': 36,
    })

    # Find maximum lengths of strings in each column
    max_lengths = find_max_lengths(df)

    # Apply formatting for columns
    apply_column_formats(worksheet, max_lengths, wrap_txt_format)

    # Apply formatting for the first column
    apply_first_column_format(worksheet, df, bold_wrap_format)
    # worksheet.write('A1', 'Test Cell', bold_wrap_format)



In [176]:
# version 1 of formatter
# def apply_sheet_formats(xlsxwriter, sheet_name, df):
#     # Workbook object
#     workbook = xlsxwriter.book
# 
#     # Get the xlsxwriter worksheet object
#     worksheet = xlsxwriter.sheets[sheet_name]
# 
#     # Create format object for text wrapping
#     wrap_txt_format = workbook.add_format({
#         'text_wrap': True,
#         'align': 'center',
#         'valign': 'vcenter',
#         'font_size': 36,
#     })
# 
#     # Create format object for bold text wrapping
#     bold_wrap_format = workbook.add_format({
#         'text_wrap': True,
#         'bold': True,
#         'align': 'center',
#         'valign': 'vcenter',
#         'font_size': 36,
#     })
# 
#     # Initialize max_length dictionary
#     max_lengths = {}
#     
#     # Loop to find max length in each column
#     for idx, col in enumerate(df.columns):
#         max_lengths[col] = max(df[col].astype(str).apply(len).max(),  # max length in column
#                                 len(str(col)))  # length of column name/header
#     
#     # Loop to set column widths
#     for idx, col in enumerate(df.columns):
#         # you might want to adjust the +1 at the end based on your specific needs
#         col_width = max_lengths[col] + 1
#         worksheet.set_column(idx+1, idx+1, col_width, wrap_txt_format)
# 
#     # Apply the bold_wrap_format to the first column
#     for row, index_value in enumerate(df.index):
#         # The length of the row label/index
#         index_length = len(str(index_value))
#         worksheet.write(row, 0, index_value, bold_wrap_format)
#         
#         # Update the width of the index (first) column
#         col_width = max(index_length, max_lengths.get(df.index.name, 0)) + 1
#         worksheet.set_column(0, 0, col_width, bold_wrap_format)


In [20]:
def get_xlsx_from_dfs(dfs):
    
    excel_file_name = get_excel_file_name()
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    with pd.ExcelWriter(excel_file_name, engine='xlsxwriter') as writer:
        # Loop through order_contents and create a sheet for each order
        for idx, df in enumerate(dfs):
            sheet_name = get_sheet_name(idx)
            
            # Write the df to sheet
            df.to_excel(writer, sheet_name, header=False, index=True)
    
            # Apply format
            apply_sheet_formats(xlsxwriter=writer, sheet_name=sheet_name, df=df)
            


In [23]:
asdf = get_xlsx_from_dfs(dfs)
# todo: 1) didn't wrap header col 2) only single sheet

Applying column formats...
Applying first column format...
Applying column formats...
Applying first column format...
Applying column formats...
Applying first column format...
Applying column formats...
Applying first column format...
Applying column formats...
Applying first column format...
Applying column formats...
Applying first column format...
Applying column formats...
Applying first column format...
Applying column formats...
Applying first column format...
Applying column formats...
Applying first column format...
Applying column formats...
Applying first column format...
Applying column formats...
Applying first column format...
Applying column formats...
Applying first column format...
Applying column formats...
Applying first column format...
Applying column formats...
Applying first column format...
Applying column formats...
Applying first column format...
Applying column formats...
Applying first column format...
Applying column formats...
Applying first column format.

In [8]:
# def get_order_id(order_text):
#     # Extract the order_id
#     order_id_start = order_text.find("Order:") + len("Order:")
#     order_id_end = order_text.find("\n", order_id_start)
#     order_id = order_text[order_id_start:order_id_end]
#     return order_id